In [ ]:
# ==============================
# Import Required Libraries
# ==============================

# Data manipulation and analysis
import pandas as pd
import numpy as np
from collections import defaultdict

# PyTorch core modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Scikit-learn utilities
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, ndcg_score

# Statistical analysis
from scipy.special import softmax
from scipy.stats import spearmanr, pearsonr

# Explainability libraries
from lime import lime_tabular
import shap

# Visualization
import matplotlib.pyplot as plt


In [ ]:
# ==============================
# Load Dataset
# ==============================

# Read the CSV file into a pandas DataFrame
# Note: Ensure that file is located in the working directory
df = pd.read_csv('iris.csv')


In [ ]:
# ==============================
# Encode Labels and Split Features/Targets
# ==============================

# Initialise a LabelEncoder to convert string labels (e.g., 'setosa', 'versicolor', 'virginica')
# into numerical labels (0, 1, 2).
le = LabelEncoder()

# Transform the last column (species names) into numeric values and store in a new column 'label'
df['label'] = le.fit_transform(df.iloc[:, -1])

# Drop the original string label column (second-to-last column) after encoding
df = df.drop(columns=[df.columns[-2]])

# Define features (X) and labels (y)
X = df.iloc[:, :-1].values   # Select all columns except 'label' → feature matrix
y = df['label'].values       # Select the 'label' column → target vector (0,1,2)


In [ ]:
# ==============================
# Train/Test Split and Data Preparation
# ==============================

# Split the dataset into training and test sets (80% / 20%).
# 'stratify=y' ensures class distribution is preserved in both sets.
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Initialise a StandardScaler and fit it on the training features.
# This standardises features (zero mean, unit variance) to improve training stability.
scaler = StandardScaler().fit(X_train_raw)

# Apply the fitted scaler to both training and test features
X_train = scaler.transform(X_train_raw)
X_test  = scaler.transform(X_test_raw)

# Convert numpy arrays into PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)   # Features → float32
y_train = torch.tensor(y_train, dtype=torch.long)      # Labels → integers (long)
X_test  = torch.tensor(X_test,  dtype=torch.float32)
y_test  = torch.tensor(y_test,  dtype=torch.long)

# Wrap tensors into TensorDataset objects and create DataLoaders
# - train_loader: batches of training data, shuffled
# - test_loader: batches of test data, not shuffled
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=16, shuffle=True)
test_loader  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=16)


In [ ]:
# ==============================
# Define Neural Network Model
# ==============================

# Create a feedforward neural network using nn.Sequential
# - Input layer: 4 features (Iris measurements)
# - Hidden layer: one fully connected (Linear) layer with ReLU activation
# - Output layer: 3 neurons (one for each Iris class)

model = nn.Sequential(
    nn.Linear(4, 8),  # Input → Hidden layer (4 inputs → 8 hidden units)
    nn.ReLU(),        # Activation function
    nn.Linear(8, 3)   # Hidden layer → Output layer (8 → 3 classes)
)

# Display the architecture of the model
print("\nModel Architecture:")
print(model)


In [ ]:
# ==============================
# Training Loop with Weight History Tracking (1 Hidden Layer)
# ==============================

# Lists to store weight/bias snapshots for each layer
w1_history, b1_history = [], []   # Hidden layer (4 → 8)
w2_history, b2_history = [], []   # Output layer (8 → 3)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 10

for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0

    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()

        # Record weights/biases after each update
        w1_history.append(model[0].weight.data.clone().cpu().numpy())  
        b1_history.append(model[0].bias.data.clone().cpu().numpy())

        w2_history.append(model[2].weight.data.clone().cpu().numpy())  
        b2_history.append(model[2].bias.data.clone().cpu().numpy())

        running_loss += loss.item()

    if epoch == 1 or epoch == epochs:
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch}/{epochs} — loss: {avg_loss:.4f}")

# Convert lists into numpy arrays for later analysis
w1_history, b1_history = np.stack(w1_history), np.stack(b1_history)
w2_history, b2_history = np.stack(w2_history), np.stack(b2_history)

print("Recorded weight history shapes:")
print(" Layer1 (4→8):", w1_history.shape)
print(" Output (8→3):", w2_history.shape)


In [ ]:
# ==============================
# Evaluate Model on Training Set
# ==============================

model.eval()  # Set model to evaluation mode (disables dropout, etc.)
train_correct, train_total = 0, 0

with torch.no_grad():  # No gradients needed during evaluation
    for xb, yb in train_loader:
        preds = model(xb).argmax(dim=1)       # Predicted class indices
        train_correct += (preds == yb).sum().item()
        train_total += yb.size(0)

# Print training accuracy
print(f"Train set accuracy: {100 * train_correct / train_total:.2f}%")


In [ ]:
# ==============================
# Evaluate Model on Test Set
# ==============================

model.eval()  # Switch to evaluation mode
correct, total = 0, 0

with torch.no_grad():  # Disable gradient tracking
    for xb, yb in test_loader:
        preds = model(xb).argmax(dim=1)   # Predicted class labels
        correct += (preds == yb).sum().item()
        total += yb.size(0)

# Print test accuracy
print(f"\nTest set accuracy: {100 * correct / total:.2f}%")


In [ ]:
# ==============================
# Analyse Weight Changes in Hidden Layer
# ==============================

# w1_history contains all snapshots of weights for the hidden layer
# Shape: (U, 8, 4) → U updates, 8 hidden units, 4 input features

# Extract the first and last snapshots
w1_init  = w1_history[0]      # Initial weights (8 × 4)
w1_final = w1_history[-1]     # Final weights   (8 × 4)

# Compute the overall weight changes across training
delta_w1 = w1_final - w1_init   # Shape: (8 × 4)

# Create a DataFrame for better readability
# Rows = hidden units, Columns = input features
feature_names = ['Sepal length', 'Sepal width', 'Petal length', 'Petal width']
hidden_units  = [f'L1 neuron {i+1}' for i in range(delta_w1.shape[0])]

df_layer1_delta = pd.DataFrame(
    delta_w1,
    index=hidden_units,
    columns=feature_names
).round(3)

# Display the changes in weights from initial to final
print("Weight (final − initial) for Hidden Layer (features → hidden, 4→8):")
print(df_layer1_delta)


In [ ]:
# ==============================
# Analyse Bias Changes in Hidden Layer
# ==============================

# b1_history contains all snapshots of biases for the hidden layer

# Extract the first and last snapshots of biases
b1_init  = b1_history[0]      # Initial biases (8,)
b1_final = b1_history[-1]     # Final biases   (8,)

# Compute the change in biases over training
delta_b1 = b1_final - b1_init   # Shape: (8,)

# Define row labels for hidden units
hidden_units = [f'L1 neuron {i+1}' for i in range(delta_b1.shape[0])]

# Create a DataFrame with one column representing bias changes
df_layer1_bias_delta = pd.DataFrame(
    delta_b1.reshape(-1, 1),
    index=hidden_units,
    columns=['Bias']
).round(3)

# Display the changes in biases
print("Bias change (final − initial) for Hidden Layer (4→8):")
print(df_layer1_bias_delta)


In [ ]:
# ==============================
# Analyse Weight Changes in Output Layer
# ==============================

# w2_history contains all snapshots of weights for the Output layer

# Extract the initial and final weight snapshots
w2_init  = w2_history[0]      # Initial weights (3 × 8)
w2_final = w2_history[-1]     # Final weights   (3 × 8)

# Compute the change in weights during training
delta_w2 = w2_final - w2_init   # Shape: (3 × 8)

# Define labels for rows and columns
classes       = ['Setosa', 'Versicolor', 'Virginica']               # Output classes
hidden_neurons = [f'L1 neuron {i+1}' for i in range(delta_w2.shape[1])]  # Hidden layer neurons

# Create a DataFrame
# Rows = Hidden layer neurons, Columns = output classes
df_output_delta = pd.DataFrame(
    delta_w2.T,
    index=hidden_neurons,
    columns=classes
).round(3)

# Display the changes in weights from Hidden → Output
print("Weight (final − initial) for Output Layer (Hidden → Output, 8→3):")
print(df_output_delta)


In [ ]:
# ==============================
# Analyse Bias Changes in Output Layer
# ==============================

# b2_history contains all snapshots of biases for the output layer

# Extract the initial and final bias snapshots for the Output layer
# Shape: (3,) → one bias per output class
b2_init  = b2_history[0]      # Initial biases (3,)
b2_final = b2_history[-1]     # Final biases   (3,)

# Compute the change in biases during training
delta_b2 = b2_final - b2_init   # Shape: (3,)

# Define labels for output neurons (corresponding to classes)
output_neurons = [f'Output neuron {i+1}' for i in range(delta_b2.shape[0])]

# Create a DataFrame with one column showing bias changes
df_output_bias_delta = pd.DataFrame(
    delta_b2.reshape(-1, 1),
    index=output_neurons,
    columns=['Bias']
).round(3)

# Display the changes in biases for the output layer
print("Bias change (final − initial) for Output Layer (Hidden → Output, 8→3):")
print(df_output_bias_delta)


In [ ]:
# ==============================
# Analytic Forward Pass Using Final Parameters (1 Hidden Layer)
# ==============================

# Final learned parameters
w1, b1 = w1_final, b1_final     # Hidden layer (8 × 4), (8,)
w2, b2 = w2_final, b2_final     # Output layer (3 × 8), (3,)

# Test data as NumPy arrays
X_test_np = X_test.numpy()      # (N, 4)
y_test_np = y_test.numpy()      # (N,)

# Analytic forward pass (ReLU activation between linear layers)
# Hidden layer: input → hidden
Z1 = X_test_np.dot(w1.T) + b1   # (N, 8)
H1 = np.maximum(0, Z1)          # ReLU

# Output layer: hidden → output logits
logits = H1.dot(w2.T) + b2      # (N, 3)
y_pred2 = np.argmax(logits, axis=1)

# Evaluation
acc = accuracy_score(y_test_np, y_pred2)
print(f"Analytic 1-hidden-layer net accuracy: {acc*100:.2f}%\n")
print(classification_report(y_test_np, y_pred2, target_names=le.classes_))


In [ ]:
# ==============================
# Region-Specific Affine Map Function (1 Hidden Layer)
# ==============================

def compute_region_affine(x, w1, b1, w2, b2):
    """
    Forward pass with ReLU mask for a 4→8→3 MLP, collapse to the region-specific affine map,
    and return the activation-pattern region id.

    Returns:
        A_r      : (3, 4)   Effective weight matrix for this region
        D_r      : (3,)     Effective bias vector for this region
        logits   : (3,)     Output logits for input x
        pred     : int      Argmax class index
        c1       : (8,)     Hidden-layer (8) neuron-level contributions to predicted class
        region_id: str      Bitstring mask, e.g., "L1:..."
    """

    # -------- Hidden layer: input (4) → hidden (8) --------
    Z1 = w1.dot(x) + b1              # (8,)
    S1 = (Z1 > 0).astype(float)      # ReLU mask (8,)
    H1 = np.maximum(0, Z1)           # (8,)

    # -------- Output layer: hidden (8) → logits (3) --------
    logits = w2.dot(H1) + b2         # (3,)
    pred   = int(np.argmax(logits))  # scalar

    # -------- Collapsed affine map (region-specific) --------
    # Mask by column-scaling w2 (equivalent to right-multiplying by diag(S1))
    W2m = w2 * S1[None, :]           # (3, 8)

    # Effective linear map from input x (4,) to logits (3,)
    A_r = W2m.dot(w1)                # (3, 4)

    # Effective bias: b2 + W2m b1
    D_r = b2 + W2m.dot(b1)           # (3,)

    # -------- Neuron-level contributions to the predicted class --------
    c1 = H1 * W2m[pred]              # (8,)

    # -------- Region ID (bitstring) --------
    S1_bits = ''.join(str(int(b)) for b in S1)   # length 8
    region_id = f"L1:{S1_bits}"

    return A_r, D_r, logits, pred, c1, region_id


In [ ]:
# ==============================
# Feature Contribution Explanation Function
# ==============================
def explain_region(x, A_r, D_r, logits, pred, feature_names, class_names):
    """
    Given affine map (A_r, D_r) for a sample, compute:
      - Logit-level feature contributions
      - Probability-level contributions via softmax Jacobian
    Returns a DataFrame with contributions per feature.
    """

    # Logit-level contributions for predicted class
    A_c = A_r[pred]                  # Row of A_r for predicted class
    f_logit = A_c * x                # Feature × coefficient

    # Compute class probabilities
    logits_r = A_r.dot(x) + D_r
    probs = softmax(logits_r)

    # Softmax Jacobian for predicted class
    J = -probs[pred] * probs
    J[pred] = probs[pred] * (1 - probs[pred])

    # Probability-level contributions
    f_all = A_r * x
    f_prob = J.dot(f_all)

    # Build DataFrame
    df = pd.DataFrame({
        'LogitContribution': f_logit,
        'ProbContribution': f_prob,
        'PredProbability': probs[pred]
    }, index=feature_names).round(4)

    return df

In [ ]:
# ==============================
# Use final parameters (1 hidden layer)
# ==============================
w1, b1 = w1_final, b1_final     # Hidden  (8 × 4), (8,)
w2, b2 = w2_final, b2_final     # Output  (3 × 8), (3,)

# ==============================
# Prepare data & names
# ==============================
feature_names = ['Sepal length', 'Sepal width', 'Petal length', 'Petal width']
class_names   = le.classes_.tolist()

# ==============================
# TRAIN set: explanations, neuron contributions, and regions
# ==============================
X_train_np = X_train.numpy()                 # (N_train, 4)
N_train, _ = X_train_np.shape

all_explanations_train = []                  # per-sample feature contributions
neuron1_train = []                           # Hidden (8,) neuron contributions per sample
rank_counts_train = {f: [0]*len(feature_names) for f in feature_names}
y_train_preds = np.zeros(N_train, dtype=int) # predictions
train_regions = {}                           # region_id -> list of train indices

for i in range(N_train):
    x = X_train_np[i]

    # Collapse to region-specific affine and get region id (1 hidden layer)
    A_r, D_r, logits, pred, c1, region_id = compute_region_affine(
        x, w1, b1, w2, b2
    )
    y_train_preds[i] = pred

    # Feature-level explanation
    df_exp = explain_region(x, A_r, D_r, logits, pred, feature_names, class_names)
    df_exp.insert(0, 'Sample', i)
    all_explanations_train.append(df_exp)

    # Rank features by absolute probability contribution
    ranking = np.argsort(-df_exp['ProbContribution'].abs().values)
    for rank, feat_idx in enumerate(ranking):
        rank_counts_train[feature_names[feat_idx]][rank] += 1

    # Store neuron contributions (hidden layer only)
    neuron1_train.append(c1)    # shape (8,)

    # Accumulate region membership
    train_regions.setdefault(region_id, []).append(i)

# Tidy outputs (optional tables)
big_df_train = (
    pd.concat(all_explanations_train)
      .reset_index().rename(columns={'index': 'Feature'})
      .set_index(['Sample','Feature'])
)
neuron1_train = np.stack(neuron1_train)      # (N_train, 8)
rank_df_train = pd.DataFrame(
    rank_counts_train,
    index=[f'Rank {r+1}' for r in range(len(feature_names))]
)

# ==============================
# TEST set: explanations, neuron contributions, and regions
# ==============================
X_test_np = X_test.numpy()                   # (N_test, 4)
N_test, _ = X_test_np.shape

all_explanations_test = []
neuron1_test = []
rank_counts_test = {f: [0]*len(feature_names) for f in feature_names}
y_test_preds = np.zeros(N_test, dtype=int)
test_regions = {}                            # region_id -> list of test indices

for i in range(N_test):
    x = X_test_np[i]

    # Collapse to region-specific affine and get region id (1 hidden layer)
    A_r, D_r, logits, pred, c1, region_id = compute_region_affine(
        x, w1, b1, w2, b2
    )
    y_test_preds[i] = pred

    # Feature-level explanation
    df_exp = explain_region(x, A_r, D_r, logits, pred, feature_names, class_names)
    df_exp.insert(0, 'Sample', i)
    all_explanations_test.append(df_exp)

    # Rank features by absolute probability contribution
    ranking = np.argsort(-df_exp['ProbContribution'].abs().values)
    for rank, feat_idx in enumerate(ranking):
        rank_counts_test[feature_names[feat_idx]][rank] += 1

    # Store neuron contributions (hidden layer only)
    neuron1_test.append(c1)                 # (8,)

    # Accumulate region membership
    test_regions.setdefault(region_id, []).append(i)

# Tidy outputs (optional tables)
big_df_test = (
    pd.concat(all_explanations_test)
      .reset_index().rename(columns={'index': 'Feature'})
      .set_index(['Sample','Feature'])
)
neuron1_test = np.stack(neuron1_test)        # (N_test, 8)
rank_df_test = pd.DataFrame(
    rank_counts_test,
    index=[f'Rank {r+1}' for r in range(len(feature_names))]
)


In [ ]:
# ==============================
# Region analysis: overlap, purity, coverage, feature profiles (TEST set) — 1 hidden layer
# ==============================

def region_analysis(
    train_regions,             # dict: region_id -> list(train idx)
    test_regions,              # dict: region_id -> list(test idx)
    X_test_np,                 # (N_test, d)
    y_test_np,                 # (N_test,)
    y_test_preds,              # (N_test,)
    feature_names,             # list[str], length d
    class_names,               # list[str], length C
    w1, b1, w2, b2,            # final parameters (4→8→3)
    purity_threshold=0.7
):
    # Overlap counts
    train_ids = set(train_regions.keys())
    test_ids  = set(test_regions.keys())
    n_regions_train = len(train_ids)
    n_regions_test  = len(test_ids)
    overlap_count   = len(train_ids & test_ids)
    test_only_count = len(test_ids - train_ids)
    overlap_pct     = overlap_count / max(1, n_regions_test)

    # Per-region class purity (TEST) uses ground truth
    C = len(class_names)
    rows = []
    for region_id, idxs in test_regions.items():
        if not idxs:
            continue
        labels = y_test_np[idxs]                      # ground-truth labels
        counts = np.bincount(labels, minlength=C)     # class histogram
        n_r = len(idxs)
        purity = (counts.max() / n_r) if n_r > 0 else 0.0
        is_confusion = (purity < purity_threshold)

        row = {'region_id': region_id, 'n_r': n_r, 'purity': purity, 'is_confusion': is_confusion}
        for c in range(C):
            row[f'count_{class_names[c]}'] = int(counts[c])
        rows.append(row)

    region_summary_test = (
        pd.DataFrame(rows)
          .set_index('region_id')
          .sort_values(by='n_r', ascending=False)
    )

    # Coverage metrics (TEST)
    N_test = len(y_test_np)
    region_sizes = sorted((len(v) for v in test_regions.values()), reverse=True)
    top1_coverage = (region_sizes[0] / N_test) if region_sizes else 0.0
    topC_coverage = (sum(region_sizes[:C]) / N_test) if region_sizes else 0.0

    # Region-wise feature profiles (TEST)
    majority_rows = []
    per_class_rows = []
    mixture_rows = []

    for region_id, idxs in test_regions.items():
        if not idxs:
            continue

        # Region affine map (any sample from region; masks define A_r, D_r)
        x0 = X_test_np[idxs[0]]
        A_r, D_r, _, _, _, _ = compute_region_affine(x0, w1, b1, w2, b2)

        # ---------- Majority class by GROUND TRUTH ----------
        labels_region = y_test_np[idxs]
        cls_idx = int(np.bincount(labels_region, minlength=C).argmax())
        cls_name = class_names[cls_idx]

        # Majority profile (use A_r[cls_idx], average over ALL samples in region)
        X_reg = X_test_np[idxs]                                  # (n_r, d)
        majority_contrib = (A_r[cls_idx][None, :] * X_reg).mean(axis=0)
        maj_row = {'region_id': region_id, 'class_used': cls_name}
        for j, feat in enumerate(feature_names):
            maj_row[feat] = float(np.round(majority_contrib[j], 4))
        majority_rows.append(maj_row)

        # ---------- Per-class profiles by GROUND TRUTH ----------
        for c in range(C):
            idxs_c = [k for k in idxs if y_test_np[k] == c]
            if not idxs_c:
                continue
            X_reg_c = X_test_np[idxs_c]                           # (n_rc, d)
            pc_contrib = (A_r[c][None, :] * X_reg_c).mean(axis=0)
            pc_row = {'region_id': region_id, 'class_used': class_names[c]}
            for j, feat in enumerate(feature_names):
                pc_row[feat] = float(np.round(pc_contrib[j], 4))
            per_class_rows.append(pc_row)

        # ---------- Mixture profile (per PREDICTED class behavior) ----------
        mix_contribs = []
        for k in idxs:
            c_pred = int(y_test_preds[k])
            mix_contribs.append(A_r[c_pred] * X_test_np[k])
        mix_profile = np.mean(np.stack(mix_contribs, axis=0), axis=0)
        mix_row = {'region_id': region_id, 'class_used': 'mixture'}
        for j, feat in enumerate(feature_names):
            mix_row[feat] = float(np.round(mix_profile[j], 4))
        mixture_rows.append(mix_row)

    # Build DataFrames
    region_feature_profile_test = (
        pd.DataFrame(majority_rows)
          .set_index('region_id')
          .sort_index()
    )
    region_feature_profile_test_per_class = (
        pd.DataFrame(per_class_rows)
          .set_index(['region_id', 'class_used'])
          .sort_index()
    )
    region_feature_profile_test_mixture = (
        pd.DataFrame(mixture_rows)
          .set_index('region_id')
          .sort_index()
    )

    # Outputs
    return {
        'n_regions_train': n_regions_train,
        'n_regions_test' : n_regions_test,
        'overlap_count'  : overlap_count,
        'test_only_count': test_only_count,
        'overlap_pct'    : overlap_pct,
        'region_summary_test'                  : region_summary_test,
        'top1_coverage'                        : top1_coverage,
        'topC_coverage'                        : topC_coverage,
        'region_feature_profile_test'          : region_feature_profile_test,              # majority (by GT)
        'region_feature_profile_test_per_class': region_feature_profile_test_per_class,    # per-class (by GT)
        'region_feature_profile_test_mixture'  : region_feature_profile_test_mixture       # mixture (by pred)
    }


In [ ]:
# ==============================
# Example usage (1 hidden layer)
# ==============================

# Final parameters
w1, b1 = w1_final, b1_final   # Hidden  (8×4), (8,)
w2, b2 = w2_final, b2_final   # Output  (3×8), (3,)

# Arrays
X_test_np = X_test.numpy()
y_test_np = y_test.numpy()

# Run analysis
results = region_analysis(
    train_regions=train_regions,
    test_regions=test_regions,
    X_test_np=X_test_np,
    y_test_np=y_test_np,
    y_test_preds=y_test_preds,
    feature_names=feature_names,
    class_names=class_names,
    w1=w1, b1=b1, w2=w2, b2=b2,
    purity_threshold=0.7
)

# Quick prints
print(f"Train regions: {results['n_regions_train']}")
print(f"Test regions: {results['n_regions_test']}")
print(f"Overlap (count): {results['overlap_count']}")
print(f"Test-only regions: {results['test_only_count']}")
print(f"Overlap (% of test regions): {results['overlap_pct']:.2%}")
print(f"Top-1 coverage (test): {results['top1_coverage']:.2%}")
print(f"Top-{len(class_names)} coverage (test): {results['topC_coverage']:.2%}")

print("\n=== Region summary (test) ===")
print(results['region_summary_test'].head(10))

print("\n=== Region-wise feature profile (majority class, test) ===")
print(results['region_feature_profile_test'].head(10))

print("\n=== Region-wise feature profile (per-class, test) ===")
print(results['region_feature_profile_test_per_class'].head(10))


In [ ]:
neuron1_matrix = neuron1_train   # or neuron1_train

for top_k in [1, 2, 3]:
    freq = np.zeros(neuron1_matrix.shape[1], dtype=int)

    for c1 in neuron1_matrix:
        top_idx = np.argsort(-np.abs(c1))[:top_k]
        freq[top_idx] += 1

    super_neuron = int(np.argmax(freq))
    print(f"Top-{top_k} super-neuron: #{super_neuron} "
          f"(appears in top-{top_k} for {freq[super_neuron]} samples)")

# Optional: active/inactive counts
active_neurons = np.sum(np.abs(neuron1_matrix) > 1e-6, axis=0) > 0
print(f"Layer 1: {active_neurons.sum()}/{len(active_neurons)} neurons active, "
      f"{(~active_neurons).sum()} inactive")


In [ ]:
neuron1_matrix = neuron1_test   # or neuron1_test

for top_k in [1, 2, 3]:
    freq = np.zeros(neuron1_matrix.shape[1], dtype=int)

    for c1 in neuron1_matrix:
        top_idx = np.argsort(-np.abs(c1))[:top_k]
        freq[top_idx] += 1

    super_neuron = int(np.argmax(freq))
    print(f"Top-{top_k} super-neuron: #{super_neuron} "
          f"(appears in top-{top_k} for {freq[super_neuron]} samples)")

# Optional: active/inactive counts
active_neurons = np.sum(np.abs(neuron1_matrix) > 1e-6, axis=0) > 0
print(f"Layer 1: {active_neurons.sum()}/{len(active_neurons)} neurons active, "
      f"{(~active_neurons).sum()} inactive")